# Instructions
- Some parts of the code are already done for you
- You need to execute all the cells
- You need to add the code where ever you see `"#### Add your code here ####"`
- Marks are mentioned along with the cells

# Face detection
Task is to predict the boundaries(mask) around the face in a given image.

## Dataset
Faces in images marked with bounding boxes. Have around 500 images with around 1100 faces manually tagged via bounding box.

### Mount Google drive if you are using google colab
- We recommend using Google Colab as you can face memory issues and longer runtimes while running on local

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

### Change current working directory to project folder (1 mark)

In [2]:
import os
#os.chdir('/home/balachandra/Desktop/Data Science/GreatLearning/Projects/Project 9 - Face Detection')
os.chdir('/content/drive/My Drive/Colab Notebooks/Face Detection')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Face Detection'

### Load the "images.npy" file (4 marks)
- This file contains images with details of bounding boxes

In [ ]:
import numpy as np
data = np.load('/content/drive/My Drive/Projects/Neural Networks/images.npy', allow_pickle = True)

In [ ]:
print(data.shape)
print(data.dtype)

### Check one sample from the loaded "images.npy" file  (4 marks)

In [ ]:
data[0:]

### Set image dimensions   (2 marks)
- Initialize image height, image width with value: 224 

In [ ]:
img_rows, img_cols = 224, 224

In [ ]:
IMAGE_WIDTH = img_rows
IMAGE_HEIGHT = img_cols

### Create features and labels
- Here feature is the image
- The label is the mask
- Images will be stored in "X_train" array
- Masks will be stored in "masks" array

In [ ]:
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input

masks = np.zeros((int(data.shape[0]), IMAGE_HEIGHT, IMAGE_WIDTH))
X_train = np.zeros((int(data.shape[0]), IMAGE_HEIGHT, IMAGE_WIDTH, 3))
for index in range(data.shape[0]):
    img = data[index][0]
    img = cv2.resize(img, dsize=(IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=cv2.INTER_CUBIC)
    try:
      img = img[:, :, :3]
    except:
      continue
    X_train[index] = preprocess_input(np.array(img, dtype=np.float32))
    for i in data[index][1]:
        x1 = int(i["points"][0]['x'] * IMAGE_WIDTH)
        x2 = int(i["points"][1]['x'] * IMAGE_WIDTH)
        y1 = int(i["points"][0]['y'] * IMAGE_HEIGHT)
        y2 = int(i["points"][1]['y'] * IMAGE_HEIGHT)
        masks[index][y1:y2, x1:x2] = 1

### Print the shape of X_train and mask array  (1 mark)

In [ ]:
print(X_train.shape)
masks.shape

### Print a sample image and image array

In [ ]:
from matplotlib import pyplot
%matplotlib inline
n = 10
print(X_train[n])
pyplot.imshow(X_train[n])

In [ ]:
pyplot.imshow(masks[n])

## Create the model (10 marks)
- Add MobileNet as model with below parameter values
  - input_shape: IMAGE_HEIGHT, IMAGE_WIDTH, 3
  - include_top: False
  - alpha: 1.0
  - weights: "imagenet"
- Add UNET architecture layers
  - This is the trickiest part of the project, you need to research and implement it correctly

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Concatenate, UpSampling2D, Conv2D, Reshape
from tensorflow.keras.models import Model

def create_model(trainable=True):
    model = MobileNet(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3), include_top=False, 
                      alpha=1.0, weights="imagenet")
    for layer in model.layers:
        layer.trainable = trainable
    
    block0 = model.get_layer("conv_pw_1_relu").output
    block1 = model.get_layer("conv_pw_1_relu").output
    block2 = model.get_layer("conv_pw_3_relu").output
    block3 = model.get_layer("conv_pw_5_relu").output
    block4 = model.get_layer("conv_pw_11_relu").output
    block5 = model.get_layer("conv_pw_13_relu").output

    x = Concatenate()([UpSampling2D()(block5), block4])
    print(x.shape)
    x = Concatenate()([UpSampling2D()(x), block3])
    print(x.shape)
    x = Concatenate()([UpSampling2D()(x), block2])
    print(x.shape)
    x = Concatenate()([UpSampling2D()(x), block1])
    print(x.shape)
    x = UpSampling2D()(x)
            
    x = Conv2D(1, kernel_size=1, activation="sigmoid")(x)
    x = Reshape((224, 224))(x)
    print(x.shape)

    return Model(inputs=model.input, outputs=x)

### Call the create_model function

In [ ]:
# Give trainable=False as argument, if you want to freeze lower layers for fast training (but low accuracy)
model = create_model()

# Print summary
model.summary()

In [ ]:
import tensorflow
tensorflow.__version__

### Define dice coefficient function (5 marks)
- Create a function to calculate dice coefficient


In [ ]:
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tensorflow.reduce_sum(y_true * y_pred)
    denominator = tensorflow.reduce_sum(y_true + y_pred)
    
    return numerator / (denominator + tensorflow.keras.backend.epsilon())

### Define loss

In [ ]:
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.backend import log, epsilon
def loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - log(dice_coefficient(y_true, y_pred) + epsilon())

### Compile the model (5 marks)
- Complie the model using below parameters
  - loss: use the loss function defined above
  - optimizers: use Adam optimizer
  - metrics: use dice_coefficient function defined above

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss=loss, optimizer=optimizer, metrics=[dice_coefficient])

In [ ]:
#### Add your code here ####

### Define checkpoint and earlystopping

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint("model-{loss:.2f}.h5", monitor="loss", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="min", period=1)
stop = EarlyStopping(monitor="loss", patience=5, mode="min")
reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1, mode="min")

### Fit the model (5 marks)
- Fit the model using below parameters
  - epochs: you can decide
  - batch_size: 1
  - callbacks: checkpoint, reduce_lr, stop

In [ ]:
model.fit(x = X_train, y = masks, batch_size = 1, epochs = 100, 
          callbacks = [checkpoint, reduce_lr, stop], verbose = 1)

### Get the predicted mask for a sample image   (5 marks)

In [ ]:
n=10
region = model.predict(np.array([X_train[n]]))

### Impose the mask on the image (5 marks)

In [ ]:
pyplot.imshow(X_train[n], 'gray', interpolation = None)
pyplot.imshow(masks[n], 'Greens', interpolation = None, alpha = 0.7)
pyplot.imshow(region[0], 'jet', interpolation = None, alpha = 0.7)

In [ ]:
n = 16
region = model.predict(np.array([X_train[n]]))
pyplot.imshow(X_train[n], 'gray', interpolation = None)
pyplot.imshow(masks[n], 'Greens', interpolation = None, alpha = 0.7)
pyplot.imshow(region[0], 'jet', interpolation = None, alpha = 0.7)

In [ ]:
n = 36
region = model.predict(np.array([X_train[n]]))
pyplot.imshow(X_train[n])
pyplot.imshow(masks[n], 'Greens', interpolation = None, alpha = 0.7)
pyplot.imshow(region[0], 'jet', alpha = 0.7)

In [ ]:
n = 360
region = model.predict(np.array([X_train[n]]))
pyplot.imshow(X_train[n])
pyplot.imshow(masks[n], 'Greens', interpolation = None, alpha = 0.7)
pyplot.imshow(region[0], 'jet', alpha = 0.7)

Model built is able to predict the faces accurately. In above plots, blue boxes represents ground truth and yellow ones overlapping the blue boxes represent predictions. In all the samples above, it can detect the faces accurately.

Same can be observed in the dice coefficeint which is around 90.67%. We can say the model is able to generalize well.